In [59]:

import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import time

# Digits dataset(d)
dig_data = load_digits()
X_digits, y_digits = dig_data.data, dig_data.target
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_digits, y_digits, test_size=0.2, random_state=42)

# Iris dataset(i)
iris_data = pd.read_csv('iris.data')
X_iris, y_iris = iris_data.iloc[:, :-1].values, iris_data.iloc[:, 4].values
encoder = LabelEncoder()
y_iris_encoded = encoder.fit_transform(y_iris)
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_iris, y_iris_encoded, test_size=0.2, random_state=42)

# Classifiers
classifier_pipelines = {
    "Perceptron": Pipeline([
        ('scaler', StandardScaler()),
        ('model', Perceptron(max_iter=1000, tol=1e-6))
    ]),
    "Logistic Regression": Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(max_iter=1000, solver='liblinear'))
    ]),
    "Linear SVM": Pipeline([
        ('scaler', StandardScaler()),
        ('model', SVC(kernel="linear"))
    ]),
    "Non-linear SVM (RBF)": Pipeline([
        ('scaler', StandardScaler()),
        ('model', SVC(kernel="rbf"))
    ]),
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": Pipeline([
        ('scaler', StandardScaler()),
        ('model', KNeighborsClassifier())
    ])
}

# Tuning hyperparameters
tuning_params = {
    "Perceptron": {'model__alpha': [0.0001, 0.001, 0.01, 0.1], 'model__eta0': [0.01, 0.1, 1]},
    "Logistic Regression": {'model__C': [0.01, 0.1, 1, 10, 100]},
    "Linear SVM": {'model__C': [0.1, 1, 10, 100]},
    "Non-linear SVM (RBF)": {'model__C': [0.1, 1, 10, 100]},
}

# Evaluation function
def evaluate_model(classifier_name, pipeline, X_train, X_test, y_train, y_test, params=None):
    start_time = time.time()
    if params:
        model = GridSearchCV(pipeline, params, cv=5)
        model.fit(X_train, y_train)
        best_params = model.best_params_
    else:
        model = pipeline
        model.fit(X_train, y_train)
        best_params = "None"

    training_time = time.time() - start_time
    accuracy_train = accuracy_score(y_train, model.predict(X_train))
    accuracy_test = accuracy_score(y_test, model.predict(X_test))

    print(f"{classifier_name}:\n"
          f"  This is Training Accuracy: {accuracy_train:.4f}\n"
          f"  This is Testing Accuracy: {accuracy_test:.4f}\n"
          f"  This is Training Time: {training_time:.4f} sec\n"
          f"  This is Best Parameter: {best_params}\n")

# Evaluating for Digits Dataset
print("Evaluating for Digits Dataset")
for name, pipe in classifier_pipelines.items():
    params = tuning_params.get(name, None)
    evaluate_model(name, pipe, X_train_d, X_test_d, y_train_d, y_test_d, params)

# Evaluating for Iris Dataset
print("\nEvaluating for Iris Dataset")
for name in ["Logistic Regression", "Non-linear SVM (RBF)", "Decision Tree"]:
    pipe = classifier_pipelines[name]
    params = tuning_params.get(name, None)
    evaluate_model(name, pipe, X_train_i, X_test_i, y_train_i, y_test_i, params)

Evaluating for Digits Dataset
Perceptron:
  This is Training Accuracy: 0.9687
  This is Testing Accuracy: 0.9500
  This is Training Time: 2.9102 sec
  This is Best Parameter: {'model__alpha': 0.0001, 'model__eta0': 0.01}

Logistic Regression:
  This is Training Accuracy: 0.9889
  This is Testing Accuracy: 0.9667
  This is Training Time: 5.9546 sec
  This is Best Parameter: {'model__C': 1}

Linear SVM:
  This is Training Accuracy: 0.9965
  This is Testing Accuracy: 0.9750
  This is Training Time: 1.1856 sec
  This is Best Parameter: {'model__C': 0.1}

Non-linear SVM (RBF):
  This is Training Accuracy: 1.0000
  This is Testing Accuracy: 0.9806
  This is Training Time: 2.4123 sec
  This is Best Parameter: {'model__C': 10}

Decision Tree:
  This is Training Accuracy: 1.0000
  This is Testing Accuracy: 0.8583
  This is Training Time: 0.0208 sec
  This is Best Parameter: Not Required

KNN:
  This is Training Accuracy: 0.9868
  This is Testing Accuracy: 0.9750
  This is Training Time: 0.0035 